In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agriculture-data/cleaned_text_data.csv


In [2]:
import pandas as pd
data = pd.read_csv('/kaggle/input/agriculture-data/cleaned_text_data.csv')

In [3]:
pip install bertopic --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 35.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.0 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver do

In [4]:
df = data.drop(columns=['file_name','date'])

In [5]:
blogs = df['text']

In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = [sent_tokenize(blog) for blog in blogs]
sentences = [sentence for doc in sentences for sentence in doc]

In [7]:
len(sentences)

628050

In [15]:
texts = pd.DataFrame(sentences, columns = ['texts'])

In [16]:
# texts.head(10)
texts.shape

(628050, 1)

In [17]:
texts.head(10)

,texts
0,With a big drop in Annual Investment Allowance...
1,100% of the cost of qualifying plant and machi...
2,Qualifying expenditure for farming and other r...
3,Saffery Champness is advising farmers that the...
4,"Martyn Dobinson, partner at the firm said: ""Wh..."
5,"""For businesses planning substantial qualifyin..."
6,"""Of course, such planning should also take int..."
7,Mr Dobinson said timing of the expenditure for...
8,"Generally, the expenditure is deemed incurred ..."
9,"""There are, however, exclusions to this rule w..."


In [19]:
texts = texts['texts']

In [23]:
texts[345]

'"Like some other nations, Canada was able to get the virus largely under control by late summer, with only a handful of new infections in major cities by August.But as the weather turns chilly, the rapid spread of new cases has been a difficult and frustrating setback.Trudeau said that his six-year-old son recently asked if Covid-19 would last forever.'

In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [ ]:
df.head(-5)

In [ ]:
df_filtered = df[(df['date'].dt.year >= 2020) & (df['date'].dt.year <= 2022)]

In [ ]:
texts = df_filtered['text'].tolist()
years = df_filtered['date'].dt.year.tolist()

In [ ]:
len(texts), len(years)

# UMAP
UMAP is an amazing technique for dimensionality reduction. In BERTopic, it is used to reduce the dimensionality of document embedding into something that is easier to use with HDBSCAN in order to create good clusters. 

We can instantiate our UMAP model and pass it to BERTopic:

In [ ]:
#pip install umap-learn --q

In [24]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(ngram_range=(2, 2), stop_words="english")
ctfidf_model = ClassTfidfTransformer()

2025-04-16 05:51:52.919318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744782713.251269      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744782713.330997      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [25]:
# Initialize the topic model
topic_model = BERTopic(
    verbose=True,
    embedding_model="paraphrase-MiniLM-L12-v2",  # Compact and fast model
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    min_topic_size=50,  # Minimum size of a topic cluster
    low_memory=True, 
    calculate_probabilities=False
)

# Fit the model on your filtered texts
topics, _ = topic_model.fit_transform(texts)

# Get number of topics (including -1 outliers)
num_topics = len(topic_model.get_topic_info())

print("Number of topics found:", num_topics)

2025-04-16 05:52:50,656 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/19627 [00:00<?, ?it/s]

2025-04-16 06:01:17,847 - BERTopic - Embedding - Completed ✓
2025-04-16 06:01:17,848 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-16 06:45:17,631 - BERTopic - Dimensionality - Completed ✓
2025-04-16 06:45:17,688 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current p

Number of topics found: 7825


In [26]:
topic_model.get_topic_info()[1:11]

,Topic,Count,Name,Representation,Representative_Docs
1,0,2515,0_invasion ukraine_russia ukraine_ukrainian fa...,"[invasion ukraine, russia ukraine, ukrainian f...","[We strive to provide national, bilateral and ..."
2,1,2159,1_organic farming_organic certification_organi...,"[organic farming, organic certification, organ...",[NPOP (National Program for Organic Production...
3,2,1200,2_covid 19_19 pandemic_impact covid_coronaviru...,"[covid 19, 19 pandemic, impact covid, coronavi...",[Final Report will add the analysis of the imp...
4,3,1132,3_potato wart_pei potato_seed potatoes_potato ...,"[potato wart, pei potato, seed potatoes, potat...",[The Canadian Food Inspection Agency (CFIA) an...
5,4,1055,4_pig farming_pig breed_raising pigs_start pig,"[pig farming, pig breed, raising pigs, start p...",[Introduction on how to start pig farming in K...
6,5,916,5_click click_want click_matthew sedacca_click...,"[click click, want click, matthew sedacca, cli...","[Click here to see more..., Click here to see ..."
7,6,901,6_gene editing_genome editing_genetic modifica...,"[gene editing, genome editing, genetic modific...","[Gene editing., What is gene editing?, What is..."
8,7,898,7_soy wax_scented candle_soy candle_scented ca...,"[soy wax, scented candle, soy candle, scented ...",[The Survive Mercury Retrograde Candle Birthda...
9,8,819,8_goat farming_goat breeds_goat breed_dairy goat,"[goat farming, goat breeds, goat breed, dairy ...","[Is goat farming profitable?, Introduction to ..."
10,9,818,9_canola futures_canola meal_canadian canola_c...,"[canola futures, canola meal, canadian canola,...",[The FCL-AGT canola crush facility will use 1....


Passing it after training allows you to fine-tune the topic representations by using .update_topics():
The great thing about using .update_topics() is that it allows you to tweak the topic representations without re-training your model! Thus, here we will be focusing on fine-tuning our topic representations after training our model.

In [32]:
# Fine-tune topic representations after training BERTopic
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3), min_df=50)
topic_model.update_topics(texts, vectorizer_model=vectorizer_model)

In [33]:
freq = topic_model.get_topic_info() 
freq.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,324710,-1_cows_meat_funding_said,"[cows, meat, funding, said, dairy, farmers, cl...",[See also: Why Australian cattle breeders have...
1,0,2515,0_ukraine_russian_russia_invasion,"[ukraine, russian, russia, invasion, war, conf...","[We strive to provide national, bilateral and ..."
2,1,2159,1_organic farming_organic_certification_pradesh,"[organic farming, organic, certification, prad...",[NPOP (National Program for Organic Production...
3,2,1200,2_covid 19_covid_coronavirus_covid 19 pandemic,"[covid 19, covid, coronavirus, covid 19 pandem...",[Final Report will add the analysis of the imp...
4,3,1132,3_potato_potatoes_prince_island,"[potato, potatoes, prince, island, bibeau, sur...",[The Canadian Food Inspection Agency (CFIA) an...
5,4,1055,4_pigs_pig_weaning_south africa,"[pigs, pig, weaning, south africa, rearing, ph...",[Introduction on how to start pig farming in K...
6,5,916,5_click_matthew_want_,"[click, matthew, want, , , , , , , ]","[Click here to see more..., Click here to see ..."
7,6,901,6_gene_dna_genes_genetic,"[gene, dna, genes, genetic, genetics, genetica...","[Gene editing., What is gene editing?, What is..."
8,7,898,7_burn_jar_courtesy_gift,"[burn, jar, courtesy, gift, pumpkin, burning, ...",[The Survive Mercury Retrograde Candle Birthda...
9,8,819,8_goat_goats_breeds_breed,"[goat, goats, breeds, breed, shelter, raising,...","[Is goat farming profitable?, Introduction to ..."


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [35]:
num_topics = len(topic_model.get_topic_info())

print("Number of topics found:", num_topics)

Number of topics found: 7825


In [34]:
topic_model.get_topic(4)

[('pigs', 0.012296005698401329),
 ('pig', 0.009760870415629369),
 ('weaning', 0.003862233240117106),
 ('south africa', 0.002966202068192822),
 ('rearing', 0.0024376538558683407),
 ('philippines', 0.0023697811710472628),
 ('breed', 0.0021666391190018207),
 ('breeds', 0.002104098371341399),
 ('raising', 0.0019172534546516326),
 ('case miss', 0.0017321873586863873)]

In [ ]:
# Make sure your timestamps are a list of years (or datetime)
timestamps = df_filtered['date'].dt.year.tolist()  # Or use full datetime if preferred

# Compute topics over time
topics_over_time = topic_model.topics_over_time(
    docs=texts,                # Use same texts from fit_transform             # Use topics returned by fit_transform
    timestamps=timestamps,     # List of years or datetimes
    global_tuning=True,
    evolution_tuning=True,
    nr_bins=20
)

In [ ]:
topics_over_time.head()

In [36]:
fig2 = topic_model.visualize_barchart(top_n_topics=9, height=700, width = 700)
fig2.write_html("topic_word_scores.html")

In [41]:
fig3 = topic_model.visualize_hierarchy(top_n_topics=50, width=700)
fig3.write_html("hierarchical_clustering.html")

In [ ]:
fig = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, height=650)
fig.write_html("topics_over_time2.html")

In [38]:
fig1 = topic_model.visualize_topics()
fig1.write_html("intertopic_distance_map2.html")

In [40]:
fig4 = topic_model.visualize_heatmap(n_clusters=20, top_n_topics=100)
fig4.write_html("similarity_matrix.html")

In [42]:
from bertopic import BERTopic 
topic_model = BERTopic() 
topic_model.save("my_model")

2025-04-16 07:16:44,230 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
# topic_model = BERTopic.load("my_model")